In [2]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")



https://github.com/microsoft/autogen/blob/main/notebook/agentchat_groupchat_stateflow.ipynb - see here for examples of coder and paper summariser agent

1. initialiser (GC manager)
2. Coder - get papers API (why is it a coder)
3. Execute the code? - human input 
4. Scientist read the papers and write a summary



In [3]:
import os
import autogen

load_dotenv() 

openai_api_key = os.getenv("OPENAI_API_KEY")


In [18]:
import json
import autogen
import os

# Get the API key from environment variable
openai_api_key = os.environ.get("OPENAI_API_KEY")

# Load the config list from JSON
config_list = autogen.config_list_from_json(
    "OA_config_list.json",
    filter_dict={"model": ["gpt-4"]}
)

# Add the API key to each config in the list
for config in config_list:
    config["api_key"] = openai_api_key

In [55]:
import tempfile
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import GroupChat
from APIs.pubmed import PubMedAPI # Import the PubMedAPI class for literature search


pubmed_api =PubMedAPI(email="sanazkazemi@hotmail.com") # for enterez to contact you if neccessary

def use_pubmed_api(query, max_results=10):
    # doc string needs to be absolutely comprehensive
    """
    Use the PubMed API to search for articles related to the given query.

    Args:
        query (str): The query string to search for.
        max_results (int): The maximum number of results to return.

    Returns:
        list: A list of article titles and abstracts.
    """
    # Search for articles related to the query
    articles = pubmed_api.search(query, max_results=max_results)

    # Extract the titles and abstracts from the articles
    results = []
    for article in articles:
        title = article.get("title", "")
        abstract = article.get("abstract", "")
        results.append({"title": title, "abstract": abstract})

    return results





temp_dir = tempfile.TemporaryDirectory()
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

gpt4_config = {
    "cache_seed": False,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

Moderator = autogen.UserProxyAgent(
    name="Moderator",
    code_execution_config=False,
    human_input_mode="NEVER",
    system_message="""
                    You are the Moderator. You receive prompts from the 
                    human and coordinate the discussion between the other 
                    agents. After receiving input from all agents, you summarize 
                    and present the final output.""",)


scientific_rational = autogen.AssistantAgent(
    name="Scientific_Rational",
    llm_config=config_list[0],
    system_message="""You provide scientific reasoning and explanations for the given 
                    prompt using articles and evidence. You reference the sources and 
                    provide a summary of the key points.""",)


safety_officer = autogen.AssistantAgent(
    name="Safety_Assistant",
    system_message="""You are the safety assistant. You ensure that the chosen drug target is safe for human use. 
                    You provide a safety assessment of the drug target and suggest any necessary modifications.",)
                     human_input_mode="NEVER""",
    llm_config=config_list[0],
)
target_assessment = autogen.AssistantAgent(
    name="Target_Assessment",
    llm_config=config_list[0],
    system_message="""You develop strategies to assess and evaluate the target or goal mentioned in the prompt. 
                    You Provide a detailed plan for the assessment and evaluation of the target.""",
)

literature_agent = autogen.AssistantAgent(
    name="Literature_Agent",
    llm_config=config_list[0],
    system_message="""You provide relevant literature and references related to the prompt. 
                    You summarize the key points and provide a list of references.""",
)

def state_transition(last_speaker, groupchat):
    if last_speaker == Moderator:
        return scientific_rational
    elif last_speaker == scientific_rational:
        return safety_officer
    elif last_speaker == safety_officer:
        return target_assessment
    elif last_speaker == target_assessment:
        return Moderator
    else:
        return None

Moderator.description = "Coordinate discussion between agents, summarize inputs, and present final output."
scientific_rational.description = "Provide scientific reasoning and explanations for the given prompt."
safety_officer.description = "Assess potential safety concerns and ethical implications related to the prompt."
target_assessment.description = "Develop strategies to assess and evaluate the target or goal mentioned in the prompt."
literature_agent.description = "Provide relevant literature and research references related to the prompt."

groupchat = autogen.GroupChat(
    agents=[Moderator, scientific_rational, safety_officer, target_assessment],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition,
    send_introductions=True,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=config_list[0])

In [56]:
chat_result = Moderator.initiate_chat(
    manager, message="what can you tell me about ACE inhibitors and their side effects? How can we assess the safety of ACE inhibitors?"
)

Moderator (to chat_manager):

what can you tell me about ACE inhibitors and their side effects? How can we assess the safety of ACE inhibitors?

--------------------------------------------------------------------------------

Next speaker: Scientific_Rational

Scientific_Rational (to chat_manager):

ACE inhibitors (Angiotensin-Converting-Enzyme inhibitors) are a class of medications mainly used to treat high blood pressure, heart failure, and other heart conditions. They work by relaxing blood vessels and reducing the volume of blood the heart must pump, thus decreasing the heart's workload.

The known side effects of ACE inhibitors include:

1. Dry cough: A persistent dry cough is a common side effect of ACE inhibitors. This occurs because ACE inhibitors cause a build-up of bradykinin, a peptide that causes blood vessels to dilate. However, in the lungs, it can stimulate cough reflexes and cause a dry cough.

2. Renal dysfunction: ACE inhibitors can lead to an increase in creatinine 

In [57]:
print(chat_result.summary)

Having heard from our team, we can outline a cohesive response:

ACE inhibitors, crucial for conditions like hypertension and heart failure, have side effects ranging from a persistent dry cough to serious issues like renal dysfunction and angioedema (Mayo Clinic, MedlinePlus). 

The Safety_Assistant stressed the need for a comprehensive patient history, possible drug interactions, dose adjustments, and the contraindication of ACE inhibitors in pregnancy. Additional safety measures include educating the patient about potential side effects and actions required in case they occur, advising against unmonitored intake of potassium supplements, coordinating with other healthcare providers involved with the patient and arranging a clear workflow for patient monitoring and follow-up. Medication safety involves healthcare providers, scientific data, and direct patient participation in full circle.

The Target_Assessment outlines a systematic strategy for evaluating ACE inhibitors safety, whic